<a href="https://colab.research.google.com/github/jhmuller/nextWord/blob/main/nextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
import string
import numpy as np
import sklearn

https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf

In [2]:
URL = 'https://www.gutenberg.org/files/67138/67138-0.txt'
response = requests.get(URL)
orig_text = response.content

In [3]:
M = 2440
temp_text = str(orig_text[M:])
clean_text = re.sub("\\\\r|\\\\n|\\\\xbb|\\\\xef|\\\\xbf|'b'", " ", temp_text)
cleaner_text = clean_text.translate(str.maketrans('', '', string.punctuation))

In [4]:
len(cleaner_text)
cleaner_text[:200]

'bBOOK I                                          CHAPTER                                     1      Robert Cohn was once middleweight boxing champion of Princeton Do not  think that I am very much imp'

In [5]:
 
N = len(cleaner_text)
cutoff = int(np.floor(0.5*N))
train = cleaner_text[:cutoff]
test = cleaner_text[cutoff:]
test[:100]

'ulls out of the cages one at  a time and they have steers in the corral to receive them and keep the'

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
def make_Xy(data, ngram_len=1):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([data])

  sequence_data = tokenizer.texts_to_sequences([data])[0]
  print(len(sequence_data))


  sequences = []
  X = []
  y = []
  for i in range(1, len(sequence_data)-ngram_len-1):
      words = sequence_data[i-1:i+ngram_len]
      X.append(words[:-1])
      y.append(words[-1])

  X = np.array(X)
  y = np.array(y)

  from  tensorflow.keras.utils import to_categorical
  vocab_size = len(tokenizer.word_index) + 1
  y = to_categorical(y, num_classes=vocab_size)
  return X, y, tokenizer

Xtrain, ytrain, TrainTokenizer = make_Xy(train, ngram_len=3)
Xtest, ytest TestTokenizer= make_Xy(test, ngram_len=3)



34953
35647


In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
def get_vocab_size(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([data])
  vocab_size = len(tokenizer.word_index) + 1
  return vocab_size

train_vocab_size = get_vocab_size(train)
model = Sequential()
model.add(Embedding(train_vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(train_vocab_size, activation="softmax"))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             43790     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 4379)              4383379   
                                                                 
Total params: 17,476,169
Trainable params: 17,476,169
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [11]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))
model.fit(Xtrain, ytrain, epochs=20, batch_size=64, callbacks=[reduce_lr, tensorboard_Visualization])

Epoch 1/20
547/547 [==============================] - 10s 14ms/step - loss: 3.7468 - lr: 0.0010
Epoch 2/20
547/547 [==============================] - 7s 14ms/step - loss: 3.4081 - lr: 0.0010
Epoch 3/20
547/547 [==============================] - 8s 14ms/step - loss: 3.1240 - lr: 0.0010
Epoch 4/20
547/547 [==============================] - 8s 14ms/step - loss: 2.8336 - lr: 0.0010
Epoch 5/20
547/547 [==============================] - 8s 14ms/step - loss: 2.5589 - lr: 0.0010
Epoch 6/20
547/547 [==============================] - 8s 14ms/step - loss: 2.3046 - lr: 0.0010
Epoch 7/20
547/547 [==============================] - 8s 14ms/step - loss: 2.0541 - lr: 0.0010
Epoch 8/20
547/547 [==============================] - 8s 14ms/step - loss: 1.8289 - lr: 0.0010
Epoch 9/20
547/547 [==============================] - 8s 14ms/step - loss: 1.6190 - lr: 0.0010
Epoch 10/20
547/547 [==============================] - 8s 14ms/step - loss: 1.4180 - lr: 0.0010
Epoch 11/20
547/547 [===========================

In [12]:
model.predict(Xtest)

1114/1114 [==============================] - 5s 4ms/step


array([[1.2267427e-14, 9.0178871e-04, 3.0979807e-06, ..., 1.2346641e-27,
        5.8442806e-27, 4.8734345e-29],
       [3.1251241e-16, 1.5305555e-10, 2.6710185e-07, ..., 8.4482007e-17,
        3.6057772e-22, 2.5465825e-09],
       [2.0148448e-24, 1.7789696e-09, 5.1117148e-08, ..., 1.2353672e-24,
        1.9465960e-35, 5.0695541e-16],
       ...,
       [0.0000000e+00, 6.8250122e-03, 1.1955472e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0094394e-16, 1.1435744e-07, 1.4351415e-06, ..., 1.0136074e-21,
        3.3032684e-28, 3.3638575e-30],
       [6.0626807e-13, 9.1553858e-04, 7.6058881e-05, ..., 1.7249037e-24,
        9.0680354e-12, 2.5774711e-25]], dtype=float32)

In [14]:
ytest.shape

(35642, 4275)